In [1]:
import xgboost as xgb
print(xgb.__version__)

3.1.2


In [2]:
import sys, xgboost as xgb
print(sys.executable)        # should point to .../.venv/bin/python
print(xgb.__version__)       # should print 3.0.4
print(xgb.__file__)          # should live under .../.venv/...

c:\Users\Paweł\Desktop\MLRegression\MLR\Scripts\python.exe
3.1.2
c:\Users\Paweł\Desktop\MLRegression\MLR\Lib\site-packages\xgboost\__init__.py


In [3]:
# ==============================================
# 1. Imports
# ==============================================
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBRegressor
import optuna
import mlflow
import mlflow.xgboost

c:\Users\Paweł\Desktop\MLRegression\MLR\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# ==============================================
# 2. Load processed datasets
# ==============================================
train_df = pd.read_csv(r"C:\Users\Paweł\Desktop\MLRegression\MLR\data\processed\feature_engineered_train.csv")
eval_df  = pd.read_csv(r"C:\Users\Paweł\Desktop\MLRegression\MLR\data\processed\feature_engineered_eval.csv")


# Define target + features
target = "price"
X_train, y_train = train_df.drop(columns=[target]), train_df[target]
X_eval, y_eval   = eval_df.drop(columns=[target]), eval_df[target]

print("Train shape:", X_train.shape)
print("Eval shape:", X_eval.shape)

Train shape: (576815, 37)
Eval shape: (148448, 37)


In [5]:
# ==============================================
# 3. Define Optuna objective function with MLflow
# ==============================================
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 200, 1000),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "gamma": trial.suggest_float("gamma", 0.0, 5.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
        "random_state": 42,
        "n_jobs": -1,
        "tree_method": "hist",
    }

    with mlflow.start_run(nested=True):
        model = XGBRegressor(**params)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_eval)
        rmse = float(np.sqrt(mean_squared_error(y_eval, y_pred)))
        mae = float(mean_absolute_error(y_eval, y_pred))
        r2 = float(r2_score(y_eval, y_pred))

        # Log hyperparameters + metrics
        mlflow.log_params(params)
        mlflow.log_metrics({"rmse": rmse, "mae": mae, "r2": r2})

    return rmse

In [10]:
# ==============================================
# 4. Run Optuna study with MLflow
# ==============================================
# Force MLflow to always use the root project mlruns folder
mlflow.set_tracking_uri("file:///C:/mlruns")
mlflow.set_experiment("xgboost_optuna_housing")

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=15)

print("Best params:", study.best_trial.params)

2025/12/02 19:02:16 INFO mlflow.tracking.fluent: Experiment with name 'xgboost_optuna_housing' does not exist. Creating a new experiment.
[I 2025-12-02 19:02:16,763] A new study created in memory with name: no-name-7bd89e72-ab47-4be1-a6d2-97710dc1ac58
[I 2025-12-02 19:02:37,566] Trial 0 finished with value: 79737.90310131577 and parameters: {'n_estimators': 962, 'max_depth': 4, 'learning_rate': 0.016651508353704898, 'subsample': 0.9654944809340773, 'colsample_bytree': 0.8255736443409329, 'min_child_weight': 4, 'gamma': 4.363731159968746, 'reg_alpha': 5.725915578930582e-07, 'reg_lambda': 0.0025306561422368122}. Best is trial 0 with value: 79737.90310131577.
[I 2025-12-02 19:02:55,194] Trial 1 finished with value: 76011.65885697241 and parameters: {'n_estimators': 309, 'max_depth': 8, 'learning_rate': 0.01122272015166067, 'subsample': 0.9777495289789793, 'colsample_bytree': 0.5467595148499765, 'min_child_weight': 1, 'gamma': 3.233097136975389, 'reg_alpha': 0.06175382654349473, 'reg_lambd

Best params: {'n_estimators': 612, 'max_depth': 8, 'learning_rate': 0.10804692179255788, 'subsample': 0.8277211554221637, 'colsample_bytree': 0.6517181778679045, 'min_child_weight': 8, 'gamma': 1.1566920845366162, 'reg_alpha': 0.01415019547777807, 'reg_lambda': 1.2022958236879162}


In [11]:
# ==============================================
# 5. Train final model with best params and log to MLflow
# ==============================================
best_params = study.best_trial.params
best_model = XGBRegressor(**best_params)
best_model.fit(X_train, y_train)

y_pred = best_model.predict(X_eval)

mae = mean_absolute_error(y_eval, y_pred)
rmse = np.sqrt(mean_squared_error(y_eval, y_pred))
r2 = r2_score(y_eval, y_pred)

print("Final tuned model performance:")
print("MAE:", mae)
print("RMSE:", rmse)
print("R²:", r2)

# Log final model
with mlflow.start_run(run_name="best_xgboost_model"):
    mlflow.log_params(best_params)
    mlflow.log_metrics({"rmse": rmse, "mae": mae, "r2": r2})
    mlflow.xgboost.log_model(best_model, name="model")

Final tuned model performance:
MAE: 31135.09323612925
RMSE: 70834.8836189962
R²: 0.9612247683916387


2025/12/02 19:15:34 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
